## Artificial Neural Network
- Theano: open source numerical computation library that can run on CPU or GPU
- Tensorflow: open source numerical computation library that can run on CPU or GPU
- Keras: builds DL NN with few lines to use above two libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
x = df.iloc[:,3:13].values
y = df.iloc[:, 13].values

In [4]:
# Encode categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_country = LabelEncoder()
x[:,1] = labelencoder_country.fit_transform(x[:,1])
labelencoder_gender = LabelEncoder()
x[:,2] = labelencoder_country.fit_transform(x[:,2])

# no relational order between the countries i.e Spain not higher than Germany
onehotencoder = OneHotEncoder(categorical_features = [1]) #index 1
x = onehotencoder.fit_transform(x).toarray()
# remove dummy variable 0,1 0,0 1,0 all represent seperate countries
x = x[:,1:]

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

In [6]:
# a lot of computations in deep learning, so its a must, don't want one indepedent variable dominating
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [7]:
import keras
from keras.models import Sequential 
from keras.layers import Dense

/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
# ANN - sequence of layers
classifier = Sequential()

1. Randomly initialize weight to small numbs
2. Input first observation of your datset in the input layer. Each feature is one input node
3. Forward-Propagation from left to right, neurons are activated in a way that the impact of each neuron's activation is limited by the weights. Propagate the activations until getting the predicted result y. 
4. Compare predicted to actual result. Measure the generated error.
5. Back Propagation: from right to left, the error is back-propagated. update the weights according to how much they are responsible for the error. The learning rate decides by how much we update the weights. 
6. Repeat 1-5 and update the weights after each observation (Reinforcement Learning) or update after only a batch of observations (Batch Learning)
7. When whole training set passed through the ANN that makes an epoch. Redo more epochs

In [9]:
# Adding input layer and first hidden layer 
# sigmoid function for probability if they leave the bank - outer layer
# rectifer function for hidden layer
# output_dim/units = number nodes in input layer is number of indepdent variables which is 11 (x.train.shape) nodes in output layer is 1 when binary outcome is 1 or 0, 11+1/2 = 6 nodes in hidden layer 
# relu = rectifier function 
# input_dim = number of indepdent variables. Havent created any layer yet, doesn't know what the nodes are expecting as inputs, so the warning comes
classifier.add(Dense(6, kernel_initializer='uniform', activation='relu', input_dim = 11))

In [10]:
# Add second hidden layer
# knows what input parameter to expect now, so input_dim is not necessary
classifier.add(Dense(6, kernel_initializer='uniform', activation='relu'))

In [11]:
# Add output layer
# output is one node, uniform for weight initialization, change activation to sigmoid for probabilities
# depedent variable with more than two categories use softmax function, this case is only two categories so sigmoid suffices
classifier.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

In [ ]:
# Compiling ANN 
# optimizer - algorithm to find most optimal set of weights
# loss - category_crossentropy for depedent variable with more than 2 categories
# metrics - only takes list 
classifier.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Fitting the ANN to the training set
classifier.fit(x_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 3s 332us/step - loss: 0.4865 - acc: 0.7957
Epoch 2/100
3310/8000 [===========>..................] - ETA: 1s - loss: 0.4265 - acc: 0.7840

In [ ]:
y_pred = classifier.predict(x_test)
# Create a threshold >0.5 return true otherwise false
y_pred = (y_pred > 0.5)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
(1515+204)/2000

Close to 86% accuracy